In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langchain langchain-community langchain-core transformers==4.52.4

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 74.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import torch
import re

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-11-07 12:56:04.660606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762520165.065862      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762520165.176985      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
purchase_schema = ResponseSchema(
    name="purchases",
    description="A list of purchased items with their names and amounts."
)
total_schema = ResponseSchema(
    name="total",
    description="The total amount of money spent on the purchases."
)

response_schemas = [purchase_schema, total_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

purchase_extraction_template = """
You are a smart assistant that extracts purchases from a user's description of their day.

Extract all purchased items and the amount spent on each, then calculate the total amount spent.

Respond ONLY in JSON format as follows:
{format_instructions}

Example Input:
"Today I bought rice for 20 dollars, then I found a good bread and I bought it for 10 dollars, and I saw candy for 30 dollars but didn’t buy it."

Now extract from the following input:
"{user_input}"
"""

In [5]:
user_input = "Today I bought rice for 20 dollars, then I found a good bread and I bought it for 10 dollars, and I saw candy for 30 dollars but didn’t buy it."

prompt = PromptTemplate(
    template=purchase_extraction_template,
    input_variables=["user_input", "format_instructions"]
).format(user_input=user_input, format_instructions=format_instructions)

response = generate_text(prompt, max_length=1000)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a smart assistant that extracts purchases from a user's description of their day.

Extract all purchased items and the amount spent on each, then calculate the total amount spent.

Respond ONLY in JSON format as follows:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"purchases": string  // A list of purchased items with their names and amounts.
	"total": string  // The total amount of money spent on the purchases.
}
```

Example Input:
"Today I bought rice for 20 dollars, then I found a good bread and I bought it for 10 dollars, and I saw candy for 30 dollars but didn’t buy it."

Now extract from the following input:
"Today I bought rice for 20 dollars, then I found a good bread and I bought it for 10 dollars, and I saw candy for 30 dollars but didn’t buy it."
```json
{
	"purchases": [
		{
			"name": "rice",
			"amount": "20"
		},
		{
			"name": "bread",
			"amount": "10"
		}


In [ ]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

In [7]:
json_text = extract_json_block(response)

In [8]:
json_text

'```json\n{\n\t"purchases": [\n\t\t{\n\t\t\t"name": "rice",\n\t\t\t"amount": "20"\n\t\t},\n\t\t{\n\t\t\t"name": "bread",\n\t\t\t"amount": "10"\n\t\t}\n\t],\n\t"total": "30"\n}\n```'

In [9]:
output_data = output_parser.parse(json_text)

In [10]:
print(output_data)

{'purchases': [{'name': 'rice', 'amount': '20'}, {'name': 'bread', 'amount': '10'}], 'total': '30'}


In [11]:
output_data['total']

'30'

In [12]:
output_data['purchases']

[{'name': 'rice', 'amount': '20'}, {'name': 'bread', 'amount': '10'}]

In [13]:
for i in range(len(output_data['purchases'])):
    product, price = output_data['purchases'][i].values()
    print(f"Product: {i+1}")
    print(f"Product: {product}")
    print(f"Price: {price}")
    print("===============================")

print(f"Totol Price is {output_data['total']}")

Product: 1
Product: rice
Price: 20
Product: 2
Product: bread
Price: 10
Totol Price is 30
